$$
Modelo
$$

In [14]:
# importar librerias necesarias
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd

In [16]:
# ingesta de datos
df_scores = pd.read_csv("ratings_final.csv", usecols=['userId','score','movieId'])
df_movie = pd.read_csv("peliculas_final.csv", usecols=['id','title'])
df_movie = df_movie.reset_index(drop=True)


### visualizar solo las columnas que usare para el modelo y la recomendacion

In [17]:
# visualizar peliculas
df_movie.head()

,title,id
0,dick johnson is dead,ns1
1,blood & water,ns2
2,ganglands,ns3
3,jailbirds new orleans,ns4
4,kota factory,ns5


In [12]:
# visualizar los ratings de peliculas por usuario
df_scores.head()

,userId,score,movieId
0,1,1.0,as680
1,1,4.5,ns2186
2,1,5.0,hs2381
3,1,5.0,ns3663
4,1,5.0,as9500


In [57]:
# Crear un objeto Reader para analizar los datos de entrada
reader = Reader(rating_scale=(0.5, 5.0))

# Cargar los datos de calificación
# ingesta de datos
df_scores = pd.read_csv("ratings_final.csv", usecols=['userId','score','movieId'])
df_movie = pd.read_csv("peliculas_final.csv", usecols=['id','title'])


### elegir un numero inferior al dataset paar entrenar el modelo y asi evitar el overfiting

In [60]:
limite = 205000
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df_scores[['userId', 'movieId','score']][:limite], reader)

### Entrenamento de modelo

In [61]:
# Dividir los datos en conjuntos de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.3, random_state=42)

# Entrenar un modelo SVD
model = SVD(n_factors=50, lr_all=0.005, reg_all=0.02, n_epochs=100)
model.fit(trainset)

In [62]:
# Hacer predicciones para un usuario específico
user_id = 1
movies_watched_by_user = df_scores[df_scores['userId'] == user_id]['movieId'].tolist()
unseen_movies_by_user = df_scores[~df_scores['movieId'].isin(movies_watched_by_user)]['movieId'].tolist()
testset = [[user_id, movie_id, 4] for movie_id in unseen_movies_by_user]
predictions = model.test(testset)

In [63]:
# Obtener las 5 películas más recomendadas
top_n = 5
recommended_movies = []
for uid, iid, _, est, _ in predictions:
    recommended_movies.append((iid, est))
recommended_movies.sort(key=lambda x: x[1], reverse=True)
recommended_movies = recommended_movies[:top_n]

In [65]:
# Obtener los títulos de las películas recomendadas
recommended_movies_titles = df_movie[df_movie['id'].isin([x[0] for x in recommended_movies])]['title'].tolist()

print('Las películas recomendadas para el usuario {} son:'.format(user_id))
for i, movie_title in enumerate(recommended_movies_titles):
    print('{}. {}'.format(i+1, movie_title))


Las películas recomendadas para el usuario 1 son:
1. bad education
2. desolation
3. don't trust the b---- in apt 23
4. girl/girl scene
